In [1]:
# === Cell 1: Locate package + (optional) schema (safe) ===
from pathlib import Path
import os, sys, shutil

NOTEBOOK_ROOT = Path.cwd().resolve()
WORK_ROOT = Path("/kaggle/working") if Path("/kaggle/working").exists() else NOTEBOOK_ROOT
PACKAGE_NAME = "arc_agi_2_submission"

def _iter_pkg(base: Path):
    if not base or not base.exists(): return
    base = base.resolve()
    if base.name == PACKAGE_NAME and (base / "kaggle_entry.py").exists():
        yield base
    for d in base.iterdir():
        if d.is_dir():
            if d.name == PACKAGE_NAME and (d / "kaggle_entry.py").exists():
                yield d.resolve()
            cand = d / PACKAGE_NAME
            if cand.exists() and (cand / "kaggle_entry.py").exists():
                yield cand.resolve()

# 1) find source package (prefer /kaggle/input)
search_roots = [Path(os.getenv("ARC_SUBMISSION_ROOT") or "")] + [
    Path("/kaggle/input"), NOTEBOOK_ROOT, NOTEBOOK_ROOT.parent, WORK_ROOT
]
submission_src = None
seen = set()
for root in filter(lambda p: p and p.exists(), search_roots):
    for cand in _iter_pkg(root):
        if cand in seen: continue
        seen.add(cand)
        if (cand / "predict_competition.py").exists():
            submission_src = cand; break
    if submission_src: break

if not submission_src:
    raise FileNotFoundError("Attach the arc_agi_2_submission dataset or set ARC_SUBMISSION_ROOT.")

submission_dst = (WORK_ROOT / PACKAGE_NAME).resolve()
if submission_src != submission_dst:
    if submission_dst.exists():
        print(f"[SYNC] Clearing {submission_dst}")
        shutil.rmtree(submission_dst)
    print(f"[SYNC] Copying {submission_src} -> {submission_dst}")
    shutil.copytree(submission_src, submission_dst)
else:
    print(f"[SYNC] Package already in working dir: {submission_dst}")

if str(submission_dst) not in sys.path:
    sys.path.insert(0, str(submission_dst))

# Optional: try to find schema, but never hard-fail
def _find_schema():
    names = ["arc-prize-2025", "ARC Prize 2025"]
    for name in names:
        for base in [submission_dst.parent, NOTEBOOK_ROOT, NOTEBOOK_ROOT.parent, Path("/kaggle/input")]:
            p = base / name
            if p.exists() and (p / "submission.schema.json").exists():
                return p.resolve()
    # last resort: search
    try:
        hit = next(Path("/kaggle/input").rglob("submission.schema.json"), None)
        return hit.parent.resolve() if hit else None
    except Exception:
        return None

spec_root = _find_schema()
print(f"[SPEC] {spec_root}" if spec_root else "[SPEC] schema not mounted (lightweight validation will be used).")


[SYNC] Copying /kaggle/input/arc-agi-2-dataset-1027/arc_agi_2_submission -> /kaggle/working/arc_agi_2_submission
[SPEC] schema not mounted (lightweight validation will be used).


In [2]:
# === Cell 2: Select RUN_SET, curate dataset dir with ONE file, override loader ===
from pathlib import Path
import os, json, shutil

# Pick what to run: "test" (submission), "evaluation", "training"
RUN_SET = os.getenv("ARC_RUN_SET", "test").lower()
CANON = {
    "test":       "arc-agi_test_challenges.json",
    "evaluation": "arc-agi_evaluation_challenges.json",
    "training":   "arc-agi_training_challenges.json",
}
if RUN_SET not in CANON:
    print(f"[WARN] Unknown ARC_RUN_SET={RUN_SET!r}; defaulting to 'test'")
    RUN_SET = "test"

# Find a source root that has the chosen file (accept hyphen/underscore variants)
SOURCE_ROOTS = [Path("/kaggle/input"), submission_dst.parent, NOTEBOOK_ROOT]
VARIANTS = {
    "test":       ["arc-agi_test_challenges.json", "arc-agi_test-challenges.json"],
    "evaluation": ["arc-agi_evaluation_challenges.json", "arc-agi_evaluation-challenges.json"],
    "training":   ["arc-agi_training_challenges.json", "arc-agi_training-challenges.json"],
}

def _find_source_file():
    for base in SOURCE_ROOTS:
        if not base.exists(): continue
        # search shallow and one directory down
        for d in [base] + [p for p in base.iterdir() if p.is_dir()]:
            for name in VARIANTS[RUN_SET]:
                p = d / name
                if p.exists():
                    return p.resolve()
    return None

src_file = _find_source_file()
if not src_file:
    raise SystemExit(
        "Could not find the chosen split.\n"
        f"RUN_SET={RUN_SET}\n"
        "Looked for any of:\n  - " + "\n  - ".join(VARIANTS[RUN_SET]) + "\n"
        "Attach a dataset such as arc-prize-2025 or arc-agi-2-public-dataset."
    )

# Read tasks into memory (normalize dict form)
with open(src_file, "r", encoding="utf-8") as f:
    blob = json.load(f)
if isinstance(blob, dict):
    selected = {str(k): v for k, v in blob.items()}
else:
    selected = {}
    for item in blob:
        if isinstance(item, dict):
            tid = item.get("task_id") or item.get("id")
            task_obj = item.get("task") or item
            if tid and isinstance(task_obj, dict):
                selected[str(tid)] = task_obj

print(f"[DISCOVER] Selected RUN_SET={RUN_SET} from {src_file} → {len(selected)} tasks")

# Curate: write ONE file in a fresh directory so the library cannot aggregate others
curated = Path("/kaggle/working/arc_curated_dataset")
if curated.exists(): shutil.rmtree(curated)
curated.mkdir(parents=True, exist_ok=True)
out_name = CANON[RUN_SET]
with open(curated / out_name, "w", encoding="utf-8") as f:
    json.dump(selected, f, ensure_ascii=False)

# (optional) tiny sample_submission to appease some loaders
if RUN_SET == "test":
    (curated / "sample_submission.json").write_text("{}", encoding="utf-8")

# Force both env + local var to the curated directory
os.environ["ARC_DATA_DIR"] = str(curated)
dataset_dir = curated
print(f"[CURATE] ARC_DATA_DIR -> {dataset_dir} (contains only {out_name})")

# === Critical: override the imported loader symbol so it ONLY returns 'selected' ===
# If you've already done: from predict_competition import load_aggregated_tasks
# we now rebind that local name here.
try:
    # If the symbol exists in globals, keep a reference (for debugging)
    _orig_load_aggregated_tasks = globals().get("load_aggregated_tasks", None)
except Exception:
    _orig_load_aggregated_tasks = None

def load_aggregated_tasks(root: Path):
    """Notebook-level override: ignore what's on disk; use curated 'selected' only."""
    # safety: ensure the caller passed the curated path
    root = Path(root)
    if root.resolve() != dataset_dir.resolve():
        print(f"[OVERRIDE] Forcing dataset_dir from {root} to {dataset_dir}")
    return dict(selected)  # shallow copy

print("[PATCH] load_aggregated_tasks() overridden to return curated 'selected' only.")


[DISCOVER] Selected RUN_SET=test from /kaggle/input/arc-prize-2025/arc-agi_test_challenges.json → 240 tasks
[CURATE] ARC_DATA_DIR -> /kaggle/working/arc_curated_dataset (contains only arc-agi_test_challenges.json)
[PATCH] load_aggregated_tasks() overridden to return curated 'selected' only.


In [3]:
# QUIET MODE: suppress side outputs & traces before running the solver
import os

# hard off for any event/trace/dump paths your package might honor
os.environ["ENABLE_EVENTS"] = "0"
os.environ["EVENT_VERBOSE"] = "0"
os.environ["ARC_ADAPTER_LOG"] = "0"
os.environ["ARC_DUMP_EXT_FIRST"] = "0"
os.environ["ARC_DEBUG_SCORES"] = "0"
os.environ["RIL_ENABLE_ENHANCEMENTS"] = os.environ.get("RIL_ENABLE_ENHANCEMENTS","1")  # keep features
os.environ["RIL_EMIT_TRACES"] = "0"
os.environ["RIL_TRACE_DIR"] = "/dev/null"
os.environ["RIL_DUMP_DIR"] = "/dev/null"
os.environ["ARC_USE_EXTERNAL"] = os.environ.get("ARC_USE_EXTERNAL","0")  # keep your choice
os.environ["ARC_EVAL_OK"] = "1"
os.environ["ARC_DISABLE_READINESS"] = "1"  # keep readiness off in Kaggle


In [4]:
# === Cell 3: Run solver on curated set and write Kaggle-compliant submission.json ===
from pathlib import Path
import os, json
import numpy as np

# Safety: ensure we only run on the curated set created in Cell 2
assert 'dataset_dir' in globals() and Path(dataset_dir).exists(), "dataset_dir missing; run Cell 1 & 2 first."
assert 'selected' in globals() and isinstance(selected, dict) and selected, "'selected' tasks missing; run Cell 2 first."
assert 'RUN_SET' in globals(), "RUN_SET missing; run Cell 2 first."
if RUN_SET != "test":
    raise SystemExit("Submission must use RUN_SET='test'. Set ARC_RUN_SET=test and re-run Cell 2.")

# --- Environment hygiene ---
os.environ.setdefault("ARC_EVAL_OK", "1")
os.environ.setdefault("ARC_DISABLE_READINESS", "1")   # keep readiness OFF in Kaggle
os.environ.setdefault("ARC_SUBMISSION_MODE", "1")
os.environ.setdefault("PYTHONHASHSEED", "1234")

# --- Import AFTER Cell 2 so our overridden load_aggregated_tasks stays in effect ---
from predict_competition import (
    load_settings,
    setup_device,
    validate_environment,
    run_ril_prediction,
)

# --- Initialize runtime ---
validate_environment()
device = setup_device()
settings = load_settings("SETTINGS.json")
if not settings:
    raise SystemExit("SETTINGS.json could not be loaded; aborting.")

# --- Use ONLY curated tasks (local override from Cell 2) ---
task_ids = sorted(selected.keys())
print(f"[TASKS] Will process {len(task_ids)} tasks (RUN_SET={RUN_SET})")
print(f"[DEVICE] {device}")

# --- Filter solver METRICS noise printed to stderr/stdout in notebooks ---
import io, sys, re, contextlib

class _DropLines(io.TextIOBase):
    """Redirect stream that drops lines beginning with certain prefixes."""
    def __init__(self, orig_stream, prefixes=("[METRICS] ",)):
        self._orig = orig_stream
        self._buf = ""
        self._pat = re.compile(rf"^({'|'.join(re.escape(p) for p in prefixes)})")

    def write(self, s):
        self._buf += s
        out = []
        while "\n" in self._buf:
            line, self._buf = self._buf.split("\n", 1)
            if not self._pat.match(line):
                out.append(line + "\n")
        if out:
            self._orig.write("".join(out))
        return len(s)

    def flush(self):
        if self._buf:
            if not self._pat.match(self._buf):
                self._orig.write(self._buf)
            self._buf = ""
        self._orig.flush()

TOPK = 2

# --- Fully quiet run_ril_prediction: hide [METRICS] noise and low-level prints ---
from IPython.utils import io as ipio
import os, sys, contextlib

@contextlib.contextmanager
def _hard_mute_fds():
    """Silence OS-level writes to stdout/stderr during the solver call."""
    devnull = os.open(os.devnull, os.O_WRONLY)
    old1, old2 = os.dup(1), os.dup(2)
    try:
        os.dup2(devnull, 1)
        os.dup2(devnull, 2)
        yield
    finally:
        os.dup2(old1, 1)
        os.dup2(old2, 2)
        os.close(old1)
        os.close(old2)
        os.close(devnull)

# Layered suppression:
#   1. Filter Python stdout/stderr lines starting with [METRICS]
#   2. Capture Jupyter display output
#   3. Mute OS-level file descriptors
with contextlib.redirect_stderr(_DropLines(sys.stderr, prefixes=("[METRICS] ",))):
    with contextlib.redirect_stdout(_DropLines(sys.stdout, prefixes=("[METRICS] ",))):
        with ipio.capture_output() as _:
            with _hard_mute_fds():
                predictions, policy_name, total_cases = run_ril_prediction(
                    task_ids, settings, topk=TOPK
                )

# ------------------------ STRICT WRITER (no sample dependency) ------------------------
# We read case counts from the curated test file in dataset_dir and emit dict-of-attempts.
def _find_test_file(curated_dir: Path) -> Path | None:
    for name in ("arc-agi_test_challenges.json", "arc-agi_test-challenges.json"):
        p = curated_dir / name
        if p.exists():
            return p
    return None

test_file = _find_test_file(Path(dataset_dir))
if not test_file:
    raise SystemExit(f"No test challenges JSON found in curated dir: {dataset_dir}")

with open(test_file, "r", encoding="utf-8") as f:
    test_blob = json.load(f)

def _normalize_test_cases(blob):
    out = {}
    if isinstance(blob, dict):
        for tid, task in blob.items():
            tests = task.get("test") or []
            out[str(tid)] = int(len(tests))
    else:
        for item in blob:
            if not isinstance(item, dict): 
                continue
            tid = item.get("task_id") or item.get("id")
            task = item.get("task") or item
            if tid and isinstance(task, dict):
                tests = task.get("test") or []
                out[str(tid)] = int(len(tests))
    return out

test_cases = _normalize_test_cases(test_blob)
test_tids = sorted(test_cases.keys())

def _to_grid_safe(grid):
    arr = np.asarray(grid, dtype=int)
    if arr.ndim == 1:
        arr = arr[None, :]
    if arr.ndim != 2 or arr.size == 0:
        return [[0]]
    if arr.shape[0] > 30 or arr.shape[1] > 30:
        arr = arr[:30, :30]
    arr = np.clip(arr, 0, 9).astype(int)
    return arr.tolist()

def _attempts_from(pred_entry):
    if isinstance(pred_entry, dict):
        items = sorted(
            pred_entry.items(),
            key=lambda kv: (int(str(kv[0]).split("_")[-1]) if str(kv[0]).split("_")[-1].isdigit() else 999, str(kv[0]))
        )
        seq = [kv[1] for kv in items]
    elif isinstance(pred_entry, list):
        seq = pred_entry
    else:
        seq = []
    return [_to_grid_safe(g) for g in seq] or [[[0]]]

TOPK = int(TOPK) if isinstance(TOPK, (int, np.integer)) else 2
submission_out = {}
missing_tasks = 0

for tid in test_tids:
    n_cases = test_cases[tid]
    pred_task = predictions.get(tid, [])
    if isinstance(pred_task, dict):
        pred_cases = [pred_task[k] for k in sorted(pred_task.keys(), key=lambda x: int(x) if str(x).isdigit() else 1_000_000)]
    elif isinstance(pred_task, list):
        pred_cases = pred_task
    else:
        pred_cases = []

    if not pred_cases:
        missing_tasks += 1

    case_objs = []
    for ci in range(n_cases):
        case_pred = pred_cases[ci] if ci < len(pred_cases) else []
        attempts = _attempts_from(case_pred)
        # enforce exactly TOPK attempts by truncating/padding
        if len(attempts) > TOPK:
            attempts = attempts[:TOPK]
        elif len(attempts) < TOPK:
            attempts += [attempts[-1]] * (TOPK - len(attempts))
        case_objs.append({f"attempt_{i+1}": g for i, g in enumerate(attempts)})

    submission_out[tid] = case_objs

with open("submission.json", "w", encoding="utf-8") as f:
    json.dump(submission_out, f, ensure_ascii=False)

print("[SUBMISSION]")
print(f"  file        : {Path('submission.json').resolve()}")
print(f"  tasks       : {len(submission_out)} (expected {len(test_tids)})")
print(f"  attempts    : top-{TOPK}")
print(f"  policy      : {policy_name}")
if missing_tasks:
    print(f"[WARN] {missing_tasks} tasks had no predictions; filled with [[0]] placeholders.")
# --------------------------------------------------------------------------------------


ℹ️  Set default values for: COMP_MODE, EVAL_MODE, NO_ADAPT, MODEL_FREEZE, DETERMINISTIC, RIL_SEED, RIL_DEVICE, ARC_ROUTER_POLICY
✅ Environment configured
💻 Using CPU
✅ Loaded settings from /kaggle/working/arc_agi_2_submission/SETTINGS.json
[TASKS] Will process 240 tasks (RUN_SET=test)
[DEVICE] cpu
[SUBMISSION]
  file        : /kaggle/working/submission.json
  tasks       : 240 (expected 240)
  attempts    : top-2
  policy      : hybrid_adapters_first


In [5]:
# CLEANUP: leave only submission.json in /kaggle/working
from pathlib import Path
import shutil, os

keep = {"submission.json"}
root = Path("/kaggle/working")

def _safe_rm(p: Path):
    try:
        if p.is_dir():
            shutil.rmtree(p, ignore_errors=True)
        else:
            p.unlink(missing_ok=True)
    except Exception:
        pass

# remove our curated dataset if it exists
curated = root / "arc_curated_dataset"
if curated.exists():
    _safe_rm(curated)

# remove copied package (optional; not needed by scorer)
pkg = root / "arc_agi_2_submission"
if pkg.exists():
    _safe_rm(pkg)

# remove everything else except submission.json
for p in root.iterdir():
    if p.name in keep:
        continue
    # ignore Kaggle system dirs that shouldn't be touched
    if p.name in {".ipynb_checkpoints"}:
        continue
    _safe_rm(p)

# final confirmation
print("[CLEANUP] Remaining files:")
for p in sorted(root.iterdir()):
    print("  -", p.name)


[CLEANUP] Remaining files:
  - submission.json
